# 2. Razvrščanje slik z nevronskimi mrežami

## Motivacija

Postopke načrtovanja, učenja in vrednotenja nevronskih mrež bomo najprej spoznali za namen razvrščanja MR slik glave v posamezne razrede, in sicer za bolnike z multiplo sklerozo. Cilj naloge bo, da le na podlagi sivinskih vrednosti slik razvrstimo bolnike v dve skupini:

- tiste z veliko prostornino in
- tiste z majhno prostornino patoloških lezij.

Motivacija za tovrstno razvrščanje pri bolnikih z multiplo sklerozo je tudi klinično motivirana. Namreč, prostornina lezij odraža breme bolezni, pri čemer večje lezije v splošnem predstavljajo večje breme bolezni za bolnika. S takim modelom lahko naprimer ob diagnozi bolezni ločimo bolnike v dve skupini, tiste s pričakovanim težjim potekom bolezni (več lezij, večja prostornina lezij) in tiste z blažjim potekom bolezni (manj lezij, manjša prostornina lezij). Prvi skupini bolnikov s tem nudimo bolj potentna zdravila že v zgodnji fazi bolezni, kar lahko upočasni razvoj bolezni.

## Programska orodja

Nevronske mreže bomo načrtovali s Python knjižnico Keras. Ta visokonivojska knjižnica predstavlja le programski vmesnik (API=application program interface) nizkonivojskih Python knjižnic za strojno učenje, med njimi najbolj popularni sta Tensorflow in Theano. Slednji knjižnici omogočata učenje model nevronskih mrež z uporabo masovno paralelnih grafičnih procesnih enot (GPU=graphic processing unit) in temeljita na klicih nižjenivojskih knjižnico kot je cuDNN in CUDA proizvajalca GPU enot NVidia. Pri tej nalogi se ne bomo ukvarjali z običajno zahtevnim nameščanjem vseh potrebnih knjižnic. Dobra navodila za ta namen dobite na spletnih straneh omenjenih knjižnic.


## Uvažanje knjižnic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Faks/AMS_colab/Analiza_slik_z_nevronskimi_mrezami
!pwd
# prompt: unzip to conent
!unzip -q /content/drive/MyDrive/Faks/AMS_colab/Analiza_slik_z_nevronskimi_mrezami/data.zip -d /content/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Faks/AMS_colab/Analiza_slik_z_nevronskimi_mrezami
/content/drive/MyDrive/Faks/AMS_colab/Analiza_slik_z_nevronskimi_mrezami


In [ ]:
!pip install SimpleITK


In [ ]:
from __future__ import print_function
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as itk
import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import Callback

from os.path import exists, join
from sklearn.model_selection import train_test_split
from amslib import load_mri_brain_data

#config = tf.compat.v1.ConfigProto
#session = tf.compat.v1.Session()

seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
CSF, GM, WM, LESIONS = 1, 2, 3, 10
TEST_DATA_FRACTION = 0.33
IMAGE_SIZE = (64, 64)
MODALITIES = ('t1',) # ('t1','flair') ali ('flair',) ali ('t1',)
NUM_CLASSES = 2 # binarno razvrščanje (lahko tudi več kategorij oz. oznak)

# 2.1 Naloži MRI podatke in loči med učne in testne
Za nalaganje podatkov bomo uporabili funkcijo load_mri_brain_data(), ki smo jo predstavili v prvem delu vaje in ki je dana v knjižnici amslib. Nato bomo z uporabo funkcije train_test_split() ustvarili učno in testno zbirki podatkov. Funkcija naključno priredi posamezno sliko in pripadajoče učne oznake v testno množico v predvidenem deležu, ki je podan s parametrom test_size. Slednji je v našem primeru določen s konstanto TEST_DATA_FRACTION. Učne podatke bomo uporabili za učenje modela, testne pa za preverjanje kakovosti rešitve oz. zmožnosti posploševanja naučenega modela.

In [ ]:
X, Y_bmsk, Y_seg = load_mri_brain_data(output_size=IMAGE_SIZE, modalities=MODALITIES, data_path = '/content/data_/')
X_train, X_test, y_train, y_test = train_test_split(X, Y_seg , test_size=TEST_DATA_FRACTION)

print('Velikost učne zbirke slik: {}'.format(X_train.shape))
print('Velikost testne zbirke slik: {}'.format(X_test.shape))

100%|██████████| 647/647 [00:43<00:00, 14.84it/s]


Velikost učne zbirke slik: (433, 64, 64, 1)
Velikost testne zbirke slik: (214, 64, 64, 1)


## Določanje mejnega praga za velikost lezij
Na osnovi danih referenčnih razgradenj v izhodu Y_seg funkcije load_mri_brain_data() bomo določili pražno vrednost velikosti lezij. Za večjo učinkovitost učenja obdelujemo le 2D rezine MR slike, iz katerih ni možno določiti celotne prostornine lezij. Zato bomo pražno vrednost določili tako, da bo število slik z majhno in veliko prostornino lezij približno enako. Enakomerna zastopanost razredov oznak (manjše/večje) običajno koristno pripomore h kakovosti naučenih modelov.

In [ ]:
# določimo mejno velikost lezij
LESION_SIZE_THRESHOLD = 30

# ANALIZA UČNEGA SETA PODATKOV
# izračunaj velikost maske lezij za vsako sliko
lesion_voxels = np.sum(np.sum(np.squeeze(y_train==LESIONS),axis=-1),axis=-1)

large_lesions = np.count_nonzero(lesion_voxels>LESION_SIZE_THRESHOLD)
small_lesions = np.count_nonzero(lesion_voxels<=LESION_SIZE_THRESHOLD)
print('Število slik z veliko prostornino lezij: {:d}'.format(large_lesions))
print('Število slik z malo prostornino lezij: {:d}'.format(small_lesions))

# pretvori vektor oznak razreda v binarno matriko oznak tipa 1-k
Y_train = keras.utils.to_categorical((lesion_voxels>LESION_SIZE_THRESHOLD).astype('int'))

Število slik z veliko prostornino lezij: 208
Število slik z malo prostornino lezij: 225


In [ ]:
# ANALIZA TESTNEGA SETA PODATKOV
# izračunaj velikost maske lezij za vsako sliko
lesion_voxels = np.sum(np.sum(np.squeeze(y_test==LESIONS),axis=-1),axis=-1)

large_lesions = np.count_nonzero(lesion_voxels>LESION_SIZE_THRESHOLD)
small_lesions = np.count_nonzero(lesion_voxels<=LESION_SIZE_THRESHOLD)
print('Število slik z veliko prostornino lezij: {:d}'.format(large_lesions))
print('Število slik z malo prostornino lezij: {:d}'.format(small_lesions))

# pretvori vektor oznak razreda v binarno matriko oznak tipa 1-k
Y_test = keras.utils.to_categorical((lesion_voxels>LESION_SIZE_THRESHOLD).astype('int'))

Število slik z veliko prostornino lezij: 78
Število slik z malo prostornino lezij: 136


# 2.2 Načrtovanje in učenje modela razvrščevalnika
## Definicija modela
Knjižnica Keras omogoča dva načina definicije modela: Sequential in Functional (model API). Prvi način je sekvenčni, kjer inicializiramo prazen model in nato dodajamo plasti sekvenčno oz. zaporedoma eno za drugo s funkcijo add(). Pri drugem pa najprej definiramo objekte posameznih gradnikov in jih poljubno (tj. ne nujno zaporedoma) povežemo med seboj. Na koncu moramo dobiti dva objekta, kjer prvi predstavlja vhod, drugi pa izhod. S tema objektoma nato inicializiramo model s klicem konstruktorja Model(inputs=, outputs=).

Pri tej nalogi bomo uporabili sekvenčni način definicije modela, pri nalogi z razgradnjo pa funkcijski način. Osnovni gradniki in njihovi parametri so na voljo s klici konstruktorjev teh gradnikov, npr.:

- Conv2D: konvolucijska plast, kjer podamo število filtrov, velikost filtrov in aktivacijsko funkcijo (v prvi plasti podamo še velikost vhodnih podatkov)
- MaxPooling: plast združevanja sosednjih odzivov, s katero zmanjšamo velikost odzivov
- Dropout: naključno ugašanje povezav modela s podanim deležem
- Dense: polno povezana plast s parametrom števila skritih nevronov in aktivacijsko funkcijo
- Flatten: pretvori poljubno vhodno polje v vektor (tipično pred Dense plastjo)

Tipično uporabljene aktivacijske funkcije so sigmoid, relu, tanh, softmax (glej možnosti aktivacijskih funkcij). S klicem funkcije summary() dobimo izpis strukture in števila parametrov modela.

In [ ]:
?Lambd

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=X_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# povzetek strukture modela in števila parametrov
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 60, 60, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,392,002 (28.20 MB)

 Trainable params: 7,392,002 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

## Hiperparametri

Nekaj pomembnih izpostavljenih parametrov, ki so vezani na učenje modela.

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 100
LEARNING_RATE = 1e-3

## Nastavitve modela za učenje
Funkcija compile() opravi prevajanje modela v strojno kodo, ki je primerna za učinkovito izvajanje. Nekateri pomembni parametri funkcije so:

- optimizer: naziv ali objekt postopka optimizacije (glej možnosti optimizers)
- loss: naziv ali objekt kriterijske funkcije (glej možnosti losses)
- metrics: seznam metrik za vrednotenje modela med učenjem in testiranjem

Pomemben hiperparameter vsakega postopka optimizacije je tudi učna konstanta (konstanta LEARNING_RATE), ki jo podamo s parametrom lr.

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), # Adadelta, RMSprop, SGD,...
              metrics=['accuracy'])

# Učenje modela
Učenje modela zaženemo s funkcijo fit(), ki ima več parametrov:

- x: numpy polje učnih podatkov
- y: numpy polje učnih oznak
- batch_size: število vzorcev za izračun gradienta uteži modela
- epochs: število ponovitev učenja, pri čemer ena epoha predstavlja posodobitve modela z uporabo vseh vhodnih podatkov
- verbose: stopnja podrobnosti izpisovanju poteka učenja (0-brez,1-prikaz napredka,2-ena vrstica na epoho)
- validation_data: par numpy polj s testnimi podatki in oznakami, za spremljanje zmožnosti posploševanja modela
- callbacks: seznam prilagojenih povratnih klicev za diagnostiko poteka učenja

Med učenjem modela lahko spremljate potek učenja z orodjem Tensorboard (osnovni meni Jupyter > New > Tensorboard, prikazno okno odprete pod Jupyter > Running > Tensorboard).

In [ ]:
# pripravi izpis kriterijskih funkcij za Tensorboard
run_count = 0
while exists('./graphs/' + str(run_count)):
    run_count += 1

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='./graphs/' + str(run_count),
    histogram_freq=0,
    write_graph=True,
    write_images=True)

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

# zaženi učenje modela
model.fit(X_train, Y_train,
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=[tbCallBack, TestCallback((X_test, Y_test))])

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.5328 - loss: 0.7028
Testing loss: 0.6667206883430481, acc: 0.6355140209197998

28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 373ms/step - accuracy: 0.5329 - loss: 0.7025 - val_accuracy: 0.6355 - val_loss: 0.6667
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.5970 - loss: 0.6713
Testing loss: 0.6516510248184204, acc: 0.7009345889091492

28/28 ━━━━━━━━━━━━━━━━━━━━ 21s 456ms/step - accuracy: 0.5956 - loss: 0.6716 - val_accuracy: 0.7009 - val_loss: 0.6517
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.6295 - loss: 0.6600
Testing loss: 0.6505507230758667, acc: 0.5934579372406006

28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 332ms/step - accuracy: 0.6295 - loss: 0.6597 - val_accuracy: 0.5935 - val_loss: 0.6506
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.6751 - loss: 0.6173
Testing loss: 0.6609986424446106, acc: 0.5841121673583984

28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 408ms/step - accuracy: 0.6745 -

## Shranjevanje modela

In [ ]:
# shrani model v lokalno mapo
model.save_weights(join('models','lesion-classification-modalities[{}].h5'.format('+'.join(MODALITIES))))
print('Model je shranjen na disk!')

ValueError: The filename must end in `.weights.h5`. Received: filepath=models/lesion-classification-modalities[t1].h5

# Primerjava vrednosti kriterijskih funkcij med učno in testno zbirko
Primerjava med kriterijskimi funkcijami na učni in testni zbirki nam omogoča zaznavanje prenasičenja učenja. Prenasičenje preprosto pomeni, da se je model razvrščevalnika specializiral za razvrščanje učnih podatkov, s čimer pa sam razvrščevalnik izgubi sposobnost posploševanja na novih, še nevidenih zbirkah podatkov.

Prenasičenje zaznamo v primeru, da se vrednosti bistveno razlikujejo med učno in testno zbirko. Za zanesljivo zaznavanje prenasičenja je smiselno pogledati tudi poteke vrednosti kriterijskih funkcij v odvisnosti od epoh. V trenutnem okolju lahko potek kriterijskih funkcij, že med učenjem modela, prikažete z orodjem Tensorboard (osnovni meni Jupyter > New > Tensorboard, prikazno okno odprete pod Jupyter > Running > Tensorboard).

In [ ]:
score = model.evaluate(X_train, Y_train, verbose=0)
print('Učna zbirka')
print('\tloss:', score[0])
print('\taccuracy:', score[1])
score = model.evaluate(X_test, Y_test, verbose=0)
print('Testna zbirka')
print('\tloss:', score[0])
print('\taccuracy:', score[1])

## Vrednotenje na posameznih slikah
Spodnja koda na testnih primerih z naučenim modelom določi oznako vhodne slike, nato prikaže vhodno sliko in pa pripadajočo razgradnjo. Izpiše se oznaka (0/1) in pa število vokslov, ki pripadajo oznaki LESIONS v referenčni razgradnji vhodne slike s pražno vrednostjo LESION_SIZE_THRESHOLD.

In [ ]:
# inicializiraj spremenljivko idx
if 'idx' not in locals():
    idx = 0

# izberi indeks testnega znaka in prikaži
idx = idx+1
idx = np.mod(idx, X_test.shape[0])
mod_img = X_test[idx,:,:,:len(MODALITIES)]
seg_img = y_test[idx,:,:,0]
_, img_rows, img_cols, num_modalities = X_test.shape

# izvedi predikcijo in prikaži oznako
p = model.predict(np.reshape(X_test[idx,:,:,:len(MODALITIES)], [1, img_rows, img_cols, num_modalities]))
print('Predikcija oznake z modelom: {:d}'.format(np.argmax(p)))
print('Število vokslov lezij (>prag): {} (>{})'.format(np.sum(seg_img==LESIONS), LESION_SIZE_THRESHOLD))
t_or_f = not np.logical_xor(np.argmax(p)>0, np.sum(seg_img==LESIONS)>LESION_SIZE_THRESHOLD)
print('=> Razvščanje {} PRAVILNO!'.format(('JE' if t_or_f else 'NI')))

# prikaži podatke
f, ax = plt.subplots(1, len(MODALITIES)+1, sharex=True, sharey=True, figsize = (20,5))
for i in range(len(MODALITIES)):
    ax[i].imshow(mod_img[:,:,i], cmap='gray')
    ax[i].set_title(MODALITIES[i].upper() + ' slika')
    ax[i].axis('off')

ax[-1].imshow(seg_img)
ax[-1].set_title('Razgradnja')
ax[-1].axis('off')

plt.show()

## Vrednotenje kakovosti razvrščanja
Kakovost razvrščanja lahko vrednotimo po različnih kriterijih in metodologijah, npr. z uporabo elementov kontingenčne tabele, izračunom izpeljanih metrik kot je občutljivost in specifičnost, z ROC (=reciever operating characteristic) krivuljami, ipd. V spodnjem primeru bomo izračunali število pravilno razvrščenih (TP+TN) in število nepravilno razvrščenih (FP+FN) testnih vzorcev in jih prikazali glede na referenčno vrednost, tj. število vokslov iz danih referenčnih razgradenj lezij.

In [ ]:
p = model.predict(X_test)
num_voxels = np.sum(
    np.reshape(y_test==LESIONS,
               (y_test.shape[0], np.prod(y_test.shape[1:]))),
    axis=-1)
p_true = np.argmax(Y_test, axis=-1)
p_est = np.argmax(p, axis=-1)
idx_true = p_est==p_true

print('Število pravilno razvrščenih (TP+TN): {}'.format(np.count_nonzero(idx_true)))
print('Število napačno razvrščenih (FP+FN): {}'.format(np.count_nonzero(~idx_true)))

plt.plot(num_voxels[idx_true], p_est[idx_true], c='g', marker='o', linewidth=0)
plt.plot(num_voxels[~idx_true], p_est[~idx_true], c='r', marker='o', linewidth=0)
plt.plot([LESION_SIZE_THRESHOLD, LESION_SIZE_THRESHOLD], [0.0, 1.0], c='k')